In [ ]:
import os
import numpy as np
import pandas as pd

from itertools import product

import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt

In [ ]:
matplotlib.rcParams.update({
    'font.size': 18,
    'legend.fontsize': 18,
    'legend.handlelength': 2
})

In [ ]:
folds = [1, 2, 3, 4, 5]

model_types = []

name_mapper = {"CNN": "Arvalus", "GCN": "D-Arvalus"}
anomaly_mapper = {"0": "Normal", "2": "Anomaly 1", "5": "Anomaly 2", "6": "Anomaly 3", "8": "Anomaly 4"}

In [ ]:
path = "../../results/calculated_scores.csv"

df_list = []

if os.path.exists(path):
    temp_df = pd.read_csv(path)
    df_list.append(temp_df)
    model_types.append(f"CNN")
    model_types.append(f"GCN")

df = pd.concat(df_list, axis = 0, ignore_index = True)

fig = None

In [ ]:
best_df = pd.DataFrame()

for fold, model_type in product(folds, model_types):
    df_ = df.loc[(df['fold'] == fold) & (df['model_type'] == model_type)]
    max_idx = df_["bc_f1_avg"].idxmax(axis=0) 
        
    best_df = best_df.append(df.loc[max_idx], ignore_index=True)
print(len(best_df))

In [ ]:
renamed_scores = {"mc_acc": "Accuracy", "mc_f1_avg": "F1", "mc_prec_avg": "Precision", "mc_rec_avg": "Recall"}
scores_to_plot = ["Accuracy", "F1", "Precision", "Recall"]

aggr = {s: ["mean"] for s in scores_to_plot}

In [ ]:
palette = ["#6ACC64", "#D65F5F", "#956CB4",
           "#8C613C", "#DC7EC0", "#797979", "#D5BB67", "#82C6E2"]

cols_to_drop = [
    "mc_acc", 
    "mc_acc_balanced",
    "mc_f1_avg", 
    "mc_f1_wavg", 
    "mc_prec_avg", 
    "mc_prec_wavg", 
    "mc_rec_avg", 
    "mc_rec_wavg", 
    "bc_acc", 
    "bc_acc_balanced",
    "bc_f1_avg", 
    "bc_f1_wavg", 
    "bc_prec_avg", 
    "bc_prec_wavg", 
    "bc_rec_avg", 
    "bc_rec_wavg", 
    "epoch"]

cols_to_drop = list(set(cols_to_drop + [col for col in list(best_df.columns) if "bc_" in col]))

rename_map = {"f1": "F1", "prec": "Precision", "rec": "Recall"}

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(12,3))

for idx, model_type in enumerate(sorted(model_types, reverse=False)):
    df_plot = best_df.loc[(best_df["model_type"] == model_type)]
    df_plot["model_type"] = df_plot["model_type"].apply(lambda val: name_mapper[val])
    df_plot = df_plot.drop(cols_to_drop, axis=1)
    df_plot = df_plot.groupby(["model_type", "fold"]).mean()
    df_plot = df_plot.stack()
    df_plot = df_plot.reset_index(level=1).reset_index()
    df_plot = df_plot.drop(["fold"], axis=1)
    df_plot.columns = ["Model", "score_type_anomaly", "score"]
    df_plot["score_type"] = df_plot["score_type_anomaly"].apply(lambda x: rename_map[x.split("_")[1]])
    df_plot["anomaly"] = df_plot["score_type_anomaly"].apply(lambda x: x.split("_")[-1])
    df_plot["anomaly"] = df_plot["anomaly"].apply(lambda val: anomaly_mapper[val])
    
    print(df_plot.groupby(["Model", "anomaly", "score_type"]).mean())
       
    sns.barplot(x="score_type", y="score", hue="anomaly", data=df_plot, ax=ax1 if idx == 0 else ax2, capsize=.075, ci="sd", palette=palette)
    if idx == 0:
        ax1.set_title("" + name_mapper[f"{model_type}"])
        ax1.get_legend().remove()
        ax1.set_xlabel("")
        ax1.set_ylabel("")
    else:
        ax2.set_title("" + name_mapper[f"{model_type}"])
        ax2.get_legend().remove()
        ax2.set_xlabel("")
        ax2.set_ylabel("")
        
plt.xlabel("")
plt.ylabel("")
plt.ylim(0, 1.05)
plt.legend(loc='upper center', ncol=3, fancybox=True, shadow=True, bbox_to_anchor=(-0.115, 1.42))
fig.savefig("../../results/plots/classification/overall_results_classes.pdf", bbox_inches='tight')